<a href="https://colab.research.google.com/github/george-kan/fastai/blob/main/Chapter_4_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 10.7MB/s 
     |████████████████████████████████| 194kB 52.6MB/s 
     |████████████████████████████████| 1.2MB 47.6MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 61kB 10.7MB/s 
Mounted at /content/gdrive


In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)

In [7]:
(path/'train/3').ls()

(#6131) [Path('/root/.fastai/data/mnist_sample/train/3/33468.png'),Path('/root/.fastai/data/mnist_sample/train/3/32753.png'),Path('/root/.fastai/data/mnist_sample/train/3/58937.png'),Path('/root/.fastai/data/mnist_sample/train/3/52307.png'),Path('/root/.fastai/data/mnist_sample/train/3/50212.png'),Path('/root/.fastai/data/mnist_sample/train/3/39384.png'),Path('/root/.fastai/data/mnist_sample/train/3/50611.png'),Path('/root/.fastai/data/mnist_sample/train/3/46570.png'),Path('/root/.fastai/data/mnist_sample/train/3/6855.png'),Path('/root/.fastai/data/mnist_sample/train/3/30229.png')...]

In [8]:
tensor(Image.open((path/'train/3').ls()[0]))

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,  19,  76, 139, 138, 170, 253, 253, 255, 253, 253, 159,  23,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   9, 130, 236, 252, 253, 252, 252, 252, 252, 253, 252, 252, 252, 211,  

The "picture" above is the internal representation of a picture. A series of 0s indicating white and various positive numbers indicating the intensity of black. For example we can see that the numbers at the edges of the picture tend to be smaller than the ones in the center.

We need to create some baseline model for classifying digits. One reasonable effort is to use the "average" 3 or the "average" 7 and compare a new digit to those two images and calculate the distance.

In [34]:
threes = [tensor(Image.open(x)) for x in (path/'train/3').ls()]
sevens = [tensor(Image.open(x)) for x in (path/'train/7').ls()]
# Here we have a list of tensors and we need the mean of all of them. The way to do this is to stack them (create a 3 dim tensor), convert them to float tensors (mean does not work on ints) and then take the mean
threes = torch.stack(threes).float()
sevens = torch.stack(sevens).float()

In [40]:
average_three = torch.mean(threes, dim = 0)
average_seven = torch.mean(sevens, dim = 0)
# The dimension specified in torch.mean is the one you collapse by. Here we had tensors of size X*28*28 where X is the number of training examples. So this is the dimension we want to collapse by

In [49]:
df = pd.DataFrame(torch.trunc(average_three))
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,4.000000,5.000000,6.000000,6.000000,6.000000,5.000000,4.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,7.000000,14.000000,23.000000,34.000000,46.000000,56.000000,62.000000,63.000000,60.000000,50.000000,38.000000,26.000000,16.000000,8.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,2.000000,7.000000,15.000000,28.000000,47.000000,71.000000,97.000000,122.000000,142.000000,156.000000,158.000000,152.000000,134.000000,107.000000,77.000000,51.000000,29.000000,13.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,1.000000,6.000000,15.000000,30.000000,52.000000,81.000000,111.000000,138.000000,160.000000,174.000000,184.000000,189.000000,189.000000,179.000000,155.000000,122.000000,86.000000,54.000000,27.000000,9.000000,2.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,2.000000,9.000000,21.000000,39.000000,61.000000,86.000000,108.000000,125.000000,134.000000,138.000000,142.000000,149.000000,160.000000,166.000000,158.000000,134.000000,103.000000,68.000000,36.000000,13.000000,2.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,3.000000,10.000000,20.000000,34.000000,49.000000,63.000000,75.000000,80.000000,80.000000,79.000000,82.000000,93.000000,115.000000,137.000000,145.000000,134.000000,106.000000,70.000000,37.000000,14.000000,2.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,2.000000,7.000000,14.000000,22.000000,30.000000,36.000000,39.000000,41.000000,41.000000,43.000000,52.000000,74.000000,106.000000,135.000000,146.000000,131.000000,99.000000,62.000000,30.000000,11.000000,2.000000,0.000000,0.000000,0.000000,0.000000


In [63]:
def dist_func(t1, bench):
  return(torch.mean((t1 - bench)**2))

In [66]:
def my_classifier(x):
  if dist_func(x, average_three) < dist_func(x, average_seven):
    return(3)
  else:
    return(7)

In [70]:
from collections import Counter
Counter([my_classifier(x) for x in threes])
Counter([my_classifier(x) for x in sevens])

Counter({3: 106, 7: 6159})

In [74]:
threes_test = [tensor(Image.open(x)) for x in (path/'valid/3').ls()]
sevens_test = [tensor(Image.open(x)) for x in (path/'valid/7').ls()]